# Econ 148 Project 3

#### Warming increases the risk of civil war in Africa

In [35]:
!pip install stargazer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from stargazer.stargazer import Stargazer

In [36]:
df_gseasSSA = pd.read_csv("gseasSSA.csv")
df_mss_countries_new = pd.read_csv("mss_countries_new.csv")
df_precip_climatology = pd.read_csv("precip_climatology.csv")
df_croparea_weight = pd.read_csv("croparea_weight.csv", encoding="windows-1252")
df_clim_conflict = pd.read_csv("clim_conflict_for_R.csv")

In [37]:
df_gseasSSA.head()

,code,country,beg_month,end_month
0,8,Angola,-2,3
1,17,Botswana,-1,4
2,26,Benin,3,8
3,34,Burundi,-3,2
4,37,Chad,5,10


In [38]:
df_mss_countries_new.head()

,country,year
0,Guinea-Bissau,1981
1,Guinea-Bissau,1982
2,Guinea-Bissau,1983
3,Guinea-Bissau,1984
4,Guinea-Bissau,1985


In [39]:
df_precip_climatology.head()

,Unnamed: 0,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,sahel,1.709537,2.492318,5.714172,16.130378,41.333336,76.099117,139.665724,164.860457,98.040216,36.330694,6.243360,2.471161
1,west_afr,8.209585,21.885972,48.073384,81.189124,128.830185,187.294634,211.482483,235.918114,212.480851,126.953568,40.818007,14.821853
2,cent_afr,150.822840,135.684500,172.917389,131.626008,66.964828,38.162833,41.202987,59.097322,81.299915,117.013004,160.153958,166.075571
3,east_afr,78.856493,77.336063,117.425652,139.091246,94.367546,57.119627,79.829072,81.796797,64.329445,63.728370,86.672489,84.657042
4,south_afr,173.127541,159.389011,126.137167,58.982098,20.821174,13.856495,13.782481,12.926650,15.509177,40.318556,91.751808,159.689238


In [40]:
df_croparea_weight.head()

,countries,item,element,2000,region
0,Angola,"Cereals,Total +",Area Harvested (Ha),890677,5.0
1,Benin,"Cereals,Total +",Area Harvested (Ha),901324,2.0
2,Botswana,"Cereals,Total +",Area Harvested (Ha),189542,5.0
3,Burkina Faso,"Cereals,Total +",Area Harvested (Ha),2661349,2.0
4,Burundi,"Cereals,Total +",Area Harvested (Ha),196300,4.0


In [41]:
df_clim_conflict.head()

,country,countryisocode,year_actual,ccode,war_prio,any_prio,war_prio_on,any_prio_on,gdp,gdp_l,...,cru_prec_diftrend,cru_prec_diftrend_lag,gdpcapgrowth,gdpcapgrowth_lag,gdpcapgrowth_lag2,gdpcap1980,polity2,polity2_lag,polity2_lag2,polity1980
0,Guinea-Bissau,GNB,1981,404,0,0,0.0,0.0,647.1996,521.2217,...,0.261924,-0.399240,24.169734,NaN,NaN,521.2217,-7,-7,-7.0,-7
1,Guinea-Bissau,GNB,1982,404,0,0,0.0,0.0,586.6083,647.1996,...,0.070554,0.261924,-9.362073,24.169734,NaN,521.2217,-7,-7,-7.0,-7
2,Guinea-Bissau,GNB,1983,404,0,0,0.0,0.0,636.3018,586.6083,...,-0.371771,0.070554,8.471326,-9.362073,24.169734,521.2217,-7,-7,-7.0,-7
3,Guinea-Bissau,GNB,1984,404,0,0,0.0,0.0,571.7829,636.3018,...,0.018248,-0.371771,-10.139670,8.471326,-9.362073,521.2217,-8,-7,-7.0,-7
4,Guinea-Bissau,GNB,1985,404,0,0,0.0,0.0,505.2217,571.7829,...,-0.035366,0.018248,-11.640992,-10.139670,8.471326,521.2217,-8,-8,-7.0,-7


In [42]:
icc_columns = [col for col in df_clim_conflict.columns if col.startswith("iccode")]
print(icc_columns)

['iccode1', 'iccode2', 'iccode3', 'iccode4', 'iccode5', 'iccode6', 'iccode7', 'iccode8', 'iccode9', 'iccode10', 'iccode11', 'iccode12', 'iccode13', 'iccode14', 'iccode15', 'iccode16', 'iccode17', 'iccode18', 'iccode19', 'iccode20', 'iccode21', 'iccode22', 'iccode23', 'iccode24', 'iccode25', 'iccode26', 'iccode27', 'iccode28', 'iccode29', 'iccode30', 'iccode31', 'iccode32', 'iccode33', 'iccode34', 'iccode35', 'iccode36', 'iccode37', 'iccode38', 'iccode39', 'iccode40', 'iccode41']


In [43]:
icc_year_columns = [col for col in df_clim_conflict.columns if col.startswith("Iccyear")]
print(icc_year_columns)

['Iccyear1', 'Iccyear2', 'Iccyear3', 'Iccyear4', 'Iccyear5', 'Iccyear6', 'Iccyear7', 'Iccyear8', 'Iccyear9', 'Iccyear10', 'Iccyear11', 'Iccyear12', 'Iccyear13', 'Iccyear14', 'Iccyear15', 'Iccyear16', 'Iccyear17', 'Iccyear18', 'Iccyear19', 'Iccyear20', 'Iccyear21', 'Iccyear22', 'Iccyear23', 'Iccyear24', 'Iccyear25', 'Iccyear26', 'Iccyear27', 'Iccyear28', 'Iccyear29', 'Iccyear30', 'Iccyear31', 'Iccyear32', 'Iccyear33', 'Iccyear34', 'Iccyear35', 'Iccyear36', 'Iccyear37', 'Iccyear38', 'Iccyear39', 'Iccyear40', 'Iccyear41']


In [44]:
#Sort Dataframe 
climate_conflict_table1 = df_clim_conflict[
    ["country"]+["year_actual"]+["war_prio_new"]+["temp_all"]+["temp_all_lag"]+
    ["prec_all"]+["prec_all_lag"]+["gdp_l"]+["polity2_lag"]+["year"]+icc_columns+icc_year_columns
]

#Drop missing values
climate_conflict_table1 = climate_conflict_table1.dropna()
climate_conflict_table1.shape

(815, 92)

In [45]:
#Define lists for the fixed effects variables in the table 1 regressions 
icc_columns_mo = [col for col in climate_conflict_table1.columns if col.startswith("iccode")]
icc_year_columns_mo = [col for col in climate_conflict_table1.columns if col.startswith("Iccyear")]

In [46]:
X1 = climate_conflict_table1[["temp_all"]+["temp_all_lag"]+icc_columns_mo+icc_year_columns_mo]
y = climate_conflict_table1["war_prio_new"]
#Add constant 
X1 = sm.add_constant(X1)
#Create model 1 regression
model1 = sm.OLS(y,X1).fit()
#Summary
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           war_prio_new   R-squared:                       0.621
Model:                            OLS   Adj. R-squared:                  0.581
Method:                 Least Squares   F-statistic:                     15.66
Date:                Fri, 25 Apr 2025   Prob (F-statistic):          1.85e-110
Time:                        20:36:46   Log-Likelihood:                 222.04
No. Observations:                 815   AIC:                            -288.1
Df Residuals:                     737   BIC:                             78.77
Df Model:                          77                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -1.3533      0.786     -1.721      0.086      -2.897       0.190
temp_all         0.0480      0.023      2.094      0.037       0.003       0.093
temp_all_lag     0.0138      0.023      0.599      0.549      -0.031       0.059
iccode1         -0.3355      0.129     -2.604      0.009      -0.588      -0.083
iccode2         -0.3222      0.144     -2.236      0.026      -0.605      -0.039
iccode3         -0.3940      0.175     -2.255      0.024      -0.737      -0.051
iccode4         -0.3703      0.164     -2.257      0.024      -0.692      -0.048
iccode5         -0.3720      0.164     -2.264      0.024      -0.695      -0.049
iccode6         -0.3534      0.157     -2.255      0.024      -0.661      -0.046
iccode7         -0.3262      0.146     -2.237      0.026      -0.613      -0.040
iccode8         -0.2719      0.125     -2.172      0.030      -0.518      -0.026
iccode9         -0.2199      0.110     -2.000      0.046      -0.436      -0.004
iccode10        -0.3971      0.177     -2.249      0.025      -0.744      -0.051
iccode11        -0.5722      0.152     -3.757      0.000      -0.871      -0.273
iccode12        -0.3652      0.116     -3.144      0.002      -0.593      -0.137
iccode13        -0.3461      0.153     -2.255      0.024      -0.647      -0.045
iccode14        -0.3310      0.147     -2.248      0.025      -0.620      -0.042
iccode15        -0.1559      0.098     -1.589      0.112      -0.348       0.037
iccode16        -0.3122      0.140     -2.231      0.026      -0.587      -0.038
iccode17        -0.1754      0.100     -1.745      0.081      -0.373       0.022
iccode18        -0.1729      0.100     -1.723      0.085      -0.370       0.024
iccode19        -0.1113      0.133     -0.836      0.404      -0.373       0.150
iccode20        -0.2413      0.098     -2.474      0.014      -0.433      -0.050
iccode21        -0.3772      0.096     -3.909      0.000      -0.567      -0.188
iccode22         1.2994      0.102     12.681      0.000       1.098       1.501
iccode23        -0.1636      0.099     -1.650      0.099      -0.358       0.031
iccode24        -0.0289      0.108     -0.267      0.790      -0.241       0.184
iccode25        -0.2600      0.149     -1.749      0.081      -0.552       0.032
iccode26         0.1072      0.182      0.589      0.556      -0.250       0.465
iccode27        -0.0416      0.136     -0.306      0.759      -0.308       0.225
iccode28     -9.546e-16   2.75e-16     -3.470      0.001   -1.49e-15   -4.15e-16
iccode29         1.2671      0.108     11.722      0.000       1.055       1.479
iccode30      3.174e-16   3.26e-16      0.975      0.330   -3.22e-16    9.57e-16
iccode31         1.3471      0.096     13.973      0.000       1.158       1.536
iccode32         0.0117      0.120      0.097      0.923      -0.224       0.247
iccod

In [47]:
#model 2
X2 = climate_conflict_table1[["temp_all"]+["temp_all_lag"]+["prec_all"]+["prec_all_lag"]+icc_columns_mo+icc_year_columns_mo]
#Add constant 
X2 = sm.add_constant(X2)
#Create model 2 regression
model2 = sm.OLS(y,X2).fit()
#Summary
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           war_prio_new   R-squared:                       0.621
Model:                            OLS   Adj. R-squared:                  0.580
Method:                 Least Squares   F-statistic:                     15.24
Date:                Fri, 25 Apr 2025   Prob (F-statistic):          2.43e-109
Time:                        20:36:46   Log-Likelihood:                 222.25
No. Observations:                 815   AIC:                            -284.5
Df Residuals:                     735   BIC:                             91.76
Df Model:                          79                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -1.3783      0.842     -1.637      0.102      -3.031       0.275
temp_all         0.0450      0.024      1.873      0.061      -0.002       0.092
temp_all_lag     0.0184      0.025      0.749      0.454      -0.030       0.067
prec_all        -0.0335      0.061     -0.546      0.585      -0.154       0.087
prec_all_lag     0.0197      0.062      0.318      0.751      -0.102       0.142
iccode1         -0.3295      0.141     -2.342      0.019      -0.606      -0.053
iccode2         -0.3273      0.145     -2.250      0.025      -0.613      -0.042
iccode3         -0.4105      0.178     -2.312      0.021      -0.759      -0.062
iccode4         -0.3802      0.165     -2.300      0.022      -0.705      -0.056
iccode5         -0.3785      0.169     -2.238      0.026      -0.711      -0.046
iccode6         -0.3711      0.163     -2.272      0.023      -0.692      -0.050
iccode7         -0.3426      0.152     -2.249      0.025      -0.642      -0.044
iccode8         -0.2724      0.133     -2.044      0.041      -0.534      -0.011
iccode9         -0.2139      0.127     -1.691      0.091      -0.462       0.034
iccode10        -0.4078      0.179     -2.282      0.023      -0.759      -0.057
iccode11        -0.5596      0.192     -2.912      0.004      -0.937      -0.182
iccode12        -0.3501      0.179     -1.953      0.051      -0.702       0.002
iccode13        -0.3500      0.161     -2.177      0.030      -0.666      -0.034
iccode14        -0.3353      0.153     -2.189      0.029      -0.636      -0.035
iccode15        -0.1497      0.111     -1.346      0.179      -0.368       0.069
iccode16        -0.3167      0.145     -2.187      0.029      -0.601      -0.032
iccode17        -0.1667      0.125     -1.332      0.183      -0.413       0.079
iccode18        -0.1694      0.107     -1.577      0.115      -0.380       0.042
iccode19        -0.1248      0.137     -0.908      0.364      -0.395       0.145
iccode20        -0.2342      0.114     -2.050      0.041      -0.458      -0.010
iccode21        -0.3711      0.108     -3.449      0.001      -0.582      -0.160
iccode22         1.3055      0.104     12.552      0.000       1.101       1.510
iccode23        -0.1685      0.101     -1.667      0.096      -0.367       0.030
iccode24        -0.0261      0.109     -0.240      0.810      -0.239       0.187
iccode25        -0.2515      0.150     -1.680      0.093      -0.545       0.042
iccode26         0.1167      0.185      0.632      0.528      -0.246       0.479
iccode27        -0.0554      0.141     -0.394      0.694      -0.332       0.221
iccode28      1.679e-15   3.68e-16      4.557      0.000    9.56e-16     2.4e-15
iccode29         1.2668      0.112     11.316      0.000       1.047       1.487
iccode30     -2.586e-15   3.32e-16     -7.781      0.000   -3.24e-15   -1.93e-15
iccod

In [48]:
X3 = climate_conflict_table1[["temp_all"]+["temp_all_lag"]+["prec_all"]+["prec_all_lag"]+["gdp_l"]+["polity2_lag"]+["year"]+
icc_columns_mo]
#Add constant 
X3 = sm.add_constant(X3)
#Create model 3 regression
model3 = sm.OLS(y,X3).fit()
#Summary
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           war_prio_new   R-squared:                       0.389
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     11.12
Date:                Fri, 25 Apr 2025   Prob (F-statistic):           9.81e-57
Time:                        20:36:46   Log-Likelihood:                 27.536
No. Observations:                 815   AIC:                             34.93
Df Residuals:                     770   BIC:                             246.6
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -1.5215      0.983     -1.547      0.122      -3.452       0.409
temp_all         0.0489      0.029      1.698      0.090      -0.008       0.106
temp_all_lag     0.0206      0.030      0.694      0.488      -0.038       0.079
prec_all         0.0165      0.075      0.221      0.825      -0.130       0.163
prec_all_lag     0.0278      0.074      0.377      0.706      -0.117       0.173
gdp_l        -2.663e-05   2.11e-05     -1.260      0.208   -6.81e-05    1.49e-05
polity2_lag     -0.0005      0.002     -0.234      0.815      -0.005       0.004
year            -0.0041      0.002     -2.169      0.030      -0.008      -0.000
iccode1         -0.3077      0.150     -2.051      0.041      -0.602      -0.013
iccode2         -0.3575      0.152     -2.358      0.019      -0.655      -0.060
iccode3         -0.4088      0.189     -2.159      0.031      -0.780      -0.037
iccode4         -0.3766      0.172     -2.185      0.029      -0.715      -0.038
iccode5         -0.3663      0.155     -2.363      0.018      -0.671      -0.062
iccode6         -0.3508      0.172     -2.041      0.042      -0.688      -0.013
iccode7         -0.3170      0.151     -2.105      0.036      -0.613      -0.021
iccode8         -0.2611      0.119     -2.196      0.028      -0.494      -0.028
iccode9         -0.2285      0.113     -2.031      0.043      -0.449      -0.008
iccode10        -0.4153      0.179     -2.327      0.020      -0.766      -0.065
iccode11        -0.1043      0.164     -0.638      0.524      -0.425       0.217
iccode12        -0.2335      0.181     -1.294      0.196      -0.588       0.121
iccode13        -0.3536      0.150     -2.351      0.019      -0.649      -0.058
iccode14        -0.3334      0.135     -2.478      0.013      -0.598      -0.069
iccode15        -0.1493      0.090     -1.665      0.096      -0.325       0.027
iccode16        -0.3248      0.131     -2.478      0.013      -0.582      -0.067
iccode17        -0.0133      0.192     -0.069      0.945      -0.391       0.364
iccode18        -0.1954      0.077     -2.550      0.011      -0.346      -0.045
iccode19        -0.2068      0.132     -1.568      0.117      -0.466       0.052
iccode20        -0.0711      0.090     -0.789      0.430      -0.248       0.106
iccode21         0.0168      0.077      0.217      0.828      -0.135       0.169
iccode22         0.3942      0.069      5.737      0.000       0.259       0.529
iccode23        -0.1473      0.064     -2.299      0.022      -0.273      -0.022
iccode24        -0.0312      0.084     -0.373      0.709      -0.195       0.133
iccode25         0.2874      0.147      1.952      0.051      -0.002       0.576
iccode26         0.3859      0.195      1.977      0.048       0.003       0.769
iccode27        -0.1077      0.131     -0.822      0.411      -0.365       0.150
iccod

In [34]:
stargazer = Stargazer([model1, model2, model3])
stargazer.custom_columns(['Model 1', 'Model 2', 'Model 3'], [1, 1, 1])
stargazer.show_model_numbers(False)
stargazer.significant_digits(4)
stargazer.covariate_order(["temp_all", "temp_all_lag", "prec_all", "prec_all_lag", "gdp_l", "polity2_lag", "const"])
stargazer.title("Table 1. Regression coefficients on climate variables, with civil war as a dependent variable")

stargazer